In [ ]:
from keras.models import load_model

import numpy as np
import librosa
import librosa.display
import sklearn
import matplotlib.pyplot as plt
import speech_recognition as VTT
from unicodedata import normalize
import os
import sys
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
pad2d = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))

In [ ]:
VTE_Array = []
TTE_Array = []

In [ ]:
CNN_model = load_model('weight.h5')

In [ ]:
for filename in os.listdir("WAV/"):
    filename = normalize('NFC', filename)
    try:
        if '.wav' not in filename in filename:
            continue
        wav, sr = librosa.load("WAV/" + filename, sr=None)
        print(filename)
        
        mfcc = librosa.feature.mfcc(y=wav, sr=sr, n_mfcc=100, n_fft=400, hop_length=160)
        mfcc = sklearn.preprocessing.scale(mfcc, axis=1)
        padded_mfcc = pad2d(mfcc, 700)
        padded_mfcc = np.expand_dims(padded_mfcc, 0)
        
        VTE_result = CNN_model.predict(padded_mfcc)
        VTE_Array.append(VTE_result)
    
    except Exception as e:
        print(filename, e)
        raise

In [ ]:
if not os.path.exists('TXT'):
    os.makedirs('TXT')

for filename in os.listdir("WAV/"):
    if not filename.endswith('.wav'):
        continue
    
    filename = normalize('NFC', filename)
    try:
        r = VTT.Recognizer()
        kr_audio = VTT.AudioFile("WAV/" + filename)
        with kr_audio as source:
            VTT_audio = r.record(source)
            
        temp = r.recognize_google(VTT_audio, language='ko-KR')
        
        # Get the file name without extension
        name = os.path.splitext(filename)[0]
        with open("TXT/" + str(name) + 'out.txt', 'w') as f:
            first_line = temp.split('\n')[0]
            print(first_line, file=f)
            
    except Exception as e:
        print(f"Error processing file {filename}: {str(e)}")

In [ ]:
rnn_model = tf.keras.models.load_model("./WtoE", custom_objects={"TextVectorization":TextVectorization})

In [ ]:
for filename in os.listdir("TXT/"):
    filename = normalize('NFC', filename)
    try:
        if '.txt' not in filename in filename:
            continue
        f = open('TXT/' + filename)
        input_txt = f.read().strip()
        f.close()
        
        emotion = float(rnn_model.predict([input_txt]))
        TTE_Array.append(emotion)
        
    except Exception as e:
        print(filename, e)
        raise

In [ ]:
TTE_Array